In [1]:
import nltk
from nltk import word_tokenize
from nltk.lm.preprocessing import padded_everygram_pipeline
from collections import Counter
from nltk.lm import Laplace, KneserNeyInterpolated, WittenBellInterpolated
import pandas as pd
import nltk
nltk.download('punkt')
import pandas as pd
from nltk import FreqDist, KneserNeyProbDist
from nltk.util import bigrams, trigrams
from nltk.util import ngrams
from nltk.lm import NgramCounter
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk import word_tokenize, sent_tokenize
from nltk.lm import MLE
from nltk.lm.api import LanguageModel, Smoothing
from nltk.lm.smoothing import AbsoluteDiscounting, KneserNey, WittenBell
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kparo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
FILE_PATH = 'data/hate/train_text.txt'

In [3]:
def tokenize_lines_list(file_path):
    line_list = []
    with open(file_path, 'r', encoding = 'utf-8') as f:
        lines = f.readlines()

    for line in lines:
        tokens = word_tokenize(line)
        line_list.append(tokens)
    
    return line_list

In [4]:
tokenized_lines = tokenize_lines_list(FILE_PATH)
training_size = (len(tokenized_lines) // 5) * 4 #80% of the original dataset

training_data = tokenized_lines[:training_size] #slice to training data
# for line in tokenized_lines[:training_size]:
#     training_data.extend(line)

c = Counter()
for line in training_data:
    c.update(line) #count words (tokens) freq count of tokens

validation_data = tokenized_lines[- (len(tokenized_lines) - training_size):] #take 20% from the end
print(len(tokenized_lines))
print(len(training_data))
print(len(validation_data))


9000
7200
1800


In [5]:
print(training_data[:1])

[['@', 'user', 'nice', 'new', 'signage', '.', 'Are', 'you', 'not', 'concerned', 'by', 'Beatlemania', '-style', 'hysterical', 'crowds', 'crongregating', 'on', 'you…']]


In [6]:
print(c.most_common(5))

[('#', 5216), ('@', 5002), ('user', 4875), ('.', 4571), ('the', 3794)]


In [7]:
n = 3

train, vocab = padded_everygram_pipeline(n, training_data)

model = KneserNeyInterpolated(n) 
model.fit(train, vocab)

In [8]:
valid = [[word for word in bigrams(line)] for line in validation_data] #for each sentece, create bigrams
valid = [sent for sent in valid if sent] #return a sentence if it's not empty. If its empty if False, and sent gets discarded
len(valid)

1799

In [9]:
from tqdm import tqdm

lst_perplexity = []
for sent in tqdm(valid[:10]):
    lst_perplexity.append(model.perplexity(sent))

100%|██████████| 10/10 [00:19<00:00,  2.00s/it]


In [10]:
pd.DataFrame(list(zip(valid, lst_perplexity)),columns = ['Trigram grouped sentences','perplexity'])

,Trigram grouped sentences,perplexity
0,"[(#, WorldCup), (WorldCup, Blog), (Blog, for),...",1136.438943
1,"[(The, ðŸ‡ªðŸ‡ºEU), (ðŸ‡ªðŸ‡ºEU, discuss), (di...",inf
2,"[(@, user), (user, Thank), (Thank, you), (you,...",inf
3,"[(@, user), (user, @), (@, user), (user, Pay),...",inf
4,"[(Prom, is), (is, coming), (coming, up), (up, ...",inf
5,"[(Sptingtime, isnt), (isnt, far), (far, .), (....",inf
6,"[(Read, World), (World, Relief), (Relief, 's),...",inf
7,"[(ICE, protester), (protester, who), (who, sca...",inf
8,"[(@, user), (user, ``), (``, 83), (83, %), (%,...",inf
9,"[(``, Differences), (Differences, are), (are, ...",inf
